In [ ]:
import stable_diffusion.stable_diffusion
# Download the files
!wget https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/main.zip
!unzip main.zip
!find continual-learning-ait-main -type f ! -name "main.ipynb" ! -name "main.py" -exec cp {} . \;

--2023-04-11 14:57:07--  https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/main [following]
--2023-04-11 14:57:07--  https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ] 154.08K  --.-KB/s    in 0.05s   

2023-04-11 14:57:07 (3.06 MB/s) - ‘main.zip’ saved [157773]

Archive:  main.zip
9d267e6b95e57ad02d2e58c6e1d1f16dcffdefcb
   creating: continual-learning-ait-main/
  i

In [1]:
import tensorflow as tf
import numpy as np
import keras
from tqdm import tqdm

import classifier
from stable_diffusion import stable_diffusion
from utils import get_grad_vector, get_future_step_parameters
from data_preparation import load_dataset, CLDataLoader

/Users/laszlofreund/miniconda/envs/cl/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
dpt_train, dpt_test = load_dataset('cifar-100')

In [3]:
train_loader = CLDataLoader(dpt_train, 64, train=True)
test_loader = CLDataLoader(dpt_train, 64, train=False)

Metal device set to: Apple M2


In [4]:
params = {
    "device": 'cuda:0' if tf.config.list_physical_devices('GPU') else 'cpu',
    "n_runs": 1,
    "n_tasks": 10,
    "n_epochs": 100,
    "n_classes": 10,
    "input_shape": (32,32, 3),
    "embedding_shape": (4,4,4),
    "samples_per_task": 100,
    "batch_size": 64,
    "num_steps": 25,

    "gen_depth": 6,
    "gen_steps": 25,
    "input_latent_strength":0.5,
    "gen_temperature":1,
    "cls_mir_gen": 1,
    "gen_mir_gen": 1,
    "cls_iters": 100,
    "cls_hiddens": 32,
    "gen_iters": 10,
    "loss_fn": "mse",

    "lr": 0.001,
    "warmup": 0,
    "max_beta": 1,
    "reuse_samples": True,
    "print_every": 5,
    "mem_coeff": 0.12,
    "n_mem": 10,
    "mir_init_prior": 10,
    "z_size": 10,
    "mir_iters": 100,
    "gen_kl_coeff": 0.5,
    "gen_rec_coeff": 0.5,
    "gen_ent_coeff": 0.5,
    "gen_div_coeff": 0.5,
    "gen_shell_coeff": 0.5,
    "cls_xent_coeff": 0.5,
    "cls_ent_coeff": 0.5,
    "cls_div_coeff": 0.5,
    "cls_shell_coeff": 0.5,
    "temperature": 1,
}

In [5]:
class Agent:
  def __init__(self, hparams):
    self.cls = None
    self.opt = None
    self.opt_gen = None
    self.gen = None
    self.params = hparams
    self.state = dict()
    self.encoder = None
    #self.decoder = None

  def set_models(self, generator=None, classifier=None):
    self.cls = classifier
    self.gen = generator
    self.encoder = generator.encoder
    #self.decoder = gen.decoder
    self.opt = tf.keras.optimizers.legacy.SGD(learning_rate=params["lr"])
    self.opt_gen = tf.keras.optimizers.legacy.Adam(learning_rate=params["lr"])

In [6]:
'''Generate samples and train the diffusion model the same time'''

def generate(agent, classifier=None, input_latent=None, train=True, coeff=1.0):

    if classifier is None:
        classifier = agent.cls

    latent, alphas, alphas_prev, timesteps = agent.gen.initialize(agent.params, input_latent)

    progbar = tqdm(list(enumerate(timesteps))[::-1])
    for index, timestep in progbar:
        progbar.set_description(f'{index:3d} {timestep:3d}')
        if train:
            with tf.GradientTape() as tape:
                e_t = agent.gen.get_model_output(
                    latent,
                    timestep,
                    agent.params['batch_size'],
                )
                a_t, a_prev = alphas[index], alphas_prev[index]
                latent = agent.gen.get_x_prev(latent, e_t,  a_t, a_prev, agent.params["temperature"])

                pred = classifier(latent)
                #loss based on confidence
                #ENT = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_pre, logits=y_pre))
                loss = coeff*tf.keras.losses.categorical_crossentropy(pred, pred)
                print("Generator loss: ", loss)
            grads = tape.gradient(loss, agent.gen.diffusion_model.trainable_variables)
            agent.opt_gen.apply_gradients(zip(grads, agent.gen.diffusion_model.trainable_variables))
        else:
            e_t = agent.gen.get_model_output(
                latent,
                timestep,
                agent.params['batch_size'],
            )
            a_t, a_prev = alphas[index], alphas_prev[index]
            latent = agent.gen.get_x_prev(latent, e_t,  a_t, a_prev, agent.params["temperature"])

    return latent

In [7]:
'''Retrive maximally interferred latent vector'''
#TODO: gradient tape

def retrieve_gen_for_cls(agent, latent):

    grad_vector = get_grad_vector(agent.cls.trainable_variables, agent.cls.grad_dims)

    virtual_cls = get_future_step_parameters(agent.cls, grad_vector, agent.cls.grad_dims, params)

    mean_latent = tf.cast(tf.reduce_mean(latent, axis=0), tf.float64)
    z_new_max = None

    for i in range(agent.params["n_mem"]):

        z_new = generate(agent, input_latent=mean_latent, train=False, coeff=0.1)

        for j in range(params["mir_iters"]):
            with tf.GradientTape(persistent=True) as tape:

                z_new = tf.cast(z_new, tf.float64)
                y_pre = agent.cls(z_new)
                y_virtual = virtual_cls(z_new)

                # maximise the interference:
                XENT = 0
                if params["cls_xent_coeff"] > 0.:
                    XENT = tf.keras.losses.categorical_crossentropy(y_virtual, y_pre)

                # the predictions from the two models should be confident
                ENT = 0
                if params["cls_ent_coeff"] > 0.:
                    ENT = tf.keras.losses.categorical_crossentropy(y_pre, y_pre)

                # the new-found samples should be different from each others
                DIV = 0
                if params["cls_div_coeff"] > 0.:
                    for found_z_i in range(i):
                        DIV += tf.keras.losses.MSE(
                            z_new,
                            z_new_max[found_z_i * z_new.shape[0]:found_z_i * z_new.shape[0] + z_new.shape[0]]
                        ) / i

                # (NEW) stay on gaussian shell loss:
                SHELL = 0
                if params["cls_shell_coeff"] > 0.:
                    SHELL = tf.keras.losses.MSE(
                        tf.norm(z_new, axis=1),
                        tf.ones_like(tf.norm(z_new, axis=1))*np.sqrt(params["z_size"])
                    )

                XENT, ENT, DIV, SHELL = tf.cast(XENT, tf.float32), tf.cast(ENT, tf.float32), tf.cast(DIV, tf.float32), tf.cast(SHELL, tf.float32)

                gain = params["cls_xent_coeff"] * XENT + \
                       -params["cls_ent_coeff"] * ENT + \
                       params["cls_div_coeff"] * DIV + \
                       -params["cls_shell_coeff"] * SHELL

            z_g = tape.gradient(gain, z_new)
            if z_g is not None:
                z_new = (z_new + 1 * z_g)

        if z_new_max is None:
            z_new_max = z_new.numpy().copy()
        else:
            z_new_max = np.concatenate([z_new_max, z_new.numpy().copy()])

    tf.stop_gradient(z_new_max)

    if np.isnan(z_new_max).any():
        mir_worked = 0
        mem_x = generate(agent, train=False)
    else:
        mem_x = z_new_max
        mir_worked = 1

    mem_y = tf.nn.softmax(agent.cls(mem_x), axis=1).numpy().copy()

    return mem_x, mem_y, mir_worked

In [8]:
#TODO: vmi más loss is

def retrieve_gen_for_gen(agent, latent):

    mean_latent = tf.cast(tf.reduce_mean(latent, axis=0), tf.float64)
    z_new_max = None


    for i in range(params["n_mem"]):

        z_new = generate(agent, input_latent=mean_latent, train=False, coeff=0.1)

        for j in range(params["mir_iters"]):

            with tf.GradientTape(persistent=True) as tape:

                # the predictions from the two models should be confident
                ENT = 0
                if params["gen_ent_coeff"]>0.:
                    y_pre = agent.cls(z_new)
                    ENT = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_pre, logits=y_pre))

                # the new-found samples should be different from each others
                DIV = 0
                if params["gen_div_coeff"]>0.:
                    for found_z_i in range(i):
                        DIV += tf.reduce_mean(tf.math.squared_difference(
                            z_new,
                            z_new_max[found_z_i * z_new.shape[0]:found_z_i * z_new.shape[0] + z_new.shape[0]])
                        ) / i

                # (NEW) stay on gaussian shell loss:
                SHELL = 0
                if params["gen_shell_coeff"]>0.:
                    SHELL = tf.reduce_mean(tf.math.squared_difference(
                        tf.norm(z_new, ord=2, axis=1),
                        tf.ones_like(tf.norm(z_new, ord=2, axis=1))*np.sqrt(params["z_size"])))


                gain =params["gen_div_coeff"] * DIV + \
                      -params["gen_ent_coeff"] * ENT + \
                       -params["gen_shell_coeff"] * SHELL

            z_g = tape.gradient(gain, z_new)
            z_new = (z_new + 1 * z_g)

        if z_new_max is None:
            z_new_max = tf.identity(z_new)
        else:
            z_new_max = tf.concat([z_new_max, z_new], axis=0)


    tf.stop_gradient(z_new_max)

    if np.isnan(z_new_max).any():
        mir_worked = 0
        mem_x = generate(agent, train=False)
    else:
        mem_x = z_new_max
        mir_worked = 1

    return mem_x, mir_worked

In [9]:
'''Train the generator unit'''

def train_generator(agent):

    data = agent.state["data"]
    latent = agent.gen.encoder(data)
    mem_x = None

    for it in range(agent.params["gen_iters"]):
        generate(agent, input_latent=latent)

        if agent.state["task"] > 0:
            if it == 0 or not agent.params["reuse_samples"]:
                print(data.shape)
                mem_x, mir_worked = retrieve_gen_for_gen(agent, latent)

                agent.state["mir_tries"] += 1
                if mir_worked:
                    agent.state["mir_success"] += 1
                    # keep for logging later
                    gen_x, gen_mem_x = data, mem_x

        if mem_x is not None:
            generate(agent, input_latent=mem_x, coeff=agent.params["mem_coeff"])


In [10]:
'''Train the encoder and the classifier unit'''

def train_classifier(agent):
    data = agent.state["data"]
    target = np.argmax(agent.state["target"], axis=1)
    mem_x, mem_y = None, None

    data_input = keras.Input(shape=agent.params["input_shape"], name="image")
    encoder_output = agent.encoder(data_input)
    cls_output = agent.cls(encoder_output)
    classifier_model = keras.Model(inputs=data_input, outputs=cls_output)
    classifier_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    for it in range(agent.params["cls_iters"]):
        classifier_model.fit(data, target, batch_size=agent.params["batch_size"], epochs=1, verbose=0)
        if agent.state["task"] > 0:
            if it == 0 or not agent.params["reuse_samples"]:
                latent = agent.gen.encoder(data)
                mem_x, mem_y, mir_worked = retrieve_gen_for_cls(agent, latent)
                agent.state["mir_tries"] += 1
                if mir_worked:
                    agent.state["mir_success"] += 1
                    # keep for logging later
                    cls_x, cls_mem_x = latent, mem_x
            if mem_x is not None:
                classifier_model.fit(mem_x, mem_y, batch_size=agent.params["batch_size"], epochs=1, verbose=0)


In [11]:
def run_epoch(agent):
  for i, (data, target) in enumerate(agent.state["tr_loader"]):
    print(data.shape)
    #print(target.shape)
    if agent.state["sample_amt"] > agent.params["samples_per_task"] > 0: break
    agent.state["sample_amt"] += data.shape[0]

    agent.state["data"] = data
    agent.state["target"] = target
    agent.state["i_example"] = i

    agent.state["beta"] = min([(agent.state["sample_amt"]) / max([agent.params["warmup"], 1.]), agent.params["max_beta"]])

    train_classifier(agent)
    train_generator(agent)

In [12]:
from copy import deepcopy
from keras.models import model_from_config

def run_task(agent):
  agent.state["sample_amt"] = 0

  for epoch in range(agent.params["n_epochs"]):
    run_epoch(agent)


  #evaluation
  #with tf.GradientTape(persistent=True) as tape:

    ''' logging

    # save some training reconstructions:
    recon_path_ = os.path.join(recon_path, f'task{agent.state["task"]}.png')
    # TODO: data
    recons = tf.concat([data.to('cpu'), x_mean.to('cpu')])
    save_image(recons, recon_path_, nrow=agent.params["batch_size"])

    # save some pretty images:
    gen_images = agent.gen.generate(25).to('cpu')
    sample_path_ = os.path.join(sample_path, f'task{agent.state["task"]}.png')
    save_image(gen_images, sample_path_, nrow=5)


    # save some MIR samples:
    if agent.state["task"] > 0:
        mir_images = tf.concat([cls_x.to('cpu'), cls_mem_x.to('cpu')])
        mir_path_ = os.path.join(mir_path, f'cls_task{task}.png')
        save_image(mir_images, mir_path_, nrow=10)

        mir_images = tf.concat([gen_x.to('cpu'), gen_mem_x.to('cpu')])
        mir_path_ = os.path.join(mir_path, f'gen_task{task}.png')
        save_image(mir_images, mir_path_, nrow=10)



    for task_t, te_loader in enumerate(test_loader):
        if task_t > agent.state["task"]: break
        LOG_temp = get_temp_logger(None, ['gen_loss', 'cls_loss', 'acc'])

 
        for i, (data, target) in enumerate(test_loader):

            logits = agent.cls.predict(data)

            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='mean')(target, logits)
            pred = logits.argmax(dim=1, keepdim=True)

            LOG_temp['acc'] += [pred.eq(target.view_as(pred)).sum().item() / pred.size(0)]
            LOG_temp['cls_loss'] += [loss.item()]

            x_mean, z_mu, z_var, ldj, z0, zk = agent.gen.generate(data)
            gen_loss, rec, kl, bpd = calculate_loss(x_mean, data, z_mu, z_var, z0,
                    zk, ldj, agent.params["input_size"], agent.params["loss_fn"], beta=agent.state["beta"])
            LOG_temp['gen_loss'] += [gen_loss.item()]



        logging_per_task(wandb, LOG, run, mode, 'acc', task, task_t,
                  np.round(np.mean(LOG_temp['acc']), 2))
        logging_per_task(wandb, LOG, run, mode, 'cls_loss', task, task_t,
                  np.round(np.mean(LOG_temp['cls_loss']), 2))
        logging_per_task(wandb, LOG, run, mode, 'gen_loss', task, task_t,
                  np.round(np.mean(LOG_temp['gen_loss']), 2))

    print(f'\n{mode}:')
    print(LOG[run][mode]['acc'])

    '''

In [13]:
def run(agent):
  agent.state["mir_tries"], agent.state["mir_success"] = 0, 0

  agent.set_models(
      classifier=classifier.classifier(agent.params),
      generator=stable_diffusion.StableDiffusion(img_height=32, img_width=32))

  for task, tr_loader in enumerate(train_loader):
    agent.state["task"] = task
    agent.state["tr_loader"] = tr_loader
    run_task(agent)

    ''' eval

      # accuracy
      final_accs = LOG[agent.state["run"]]['acc'][:, agent.state["task"]]
      logging_per_task(wandb, LOG, run, mode, 'final_acc', task, value=np.round(np.mean(final_accs),2))

      # forgetting
      best_acc = np.max(LOG[run][mode]['acc'], 1)
      final_forgets = best_acc - LOG[run][mode]['acc'][:, task]
      logging_per_task(wandb, LOG, run, mode, 'final_forget', task, value=np.round(np.mean(final_forgets[:-1]),2))

      # VAE loss
      final_elbos = LOG[run][mode]['gen_loss'][:, task]
      logging_per_task(wandb, LOG, run, mode, 'final_elbo', task, value=np.round(np.mean(final_elbos), 2))

      print(f'\n{mode}:')
      print(f'final accuracy: {final_accs}')
      print(f'average: {LOG[run][mode]["final_acc"]}')
      print(f'final forgetting: {final_forgets}')
      print(f'average: {LOG[run][mode]["final_forget"]}')
      print(f'final VAE loss: {final_elbos}')
      print(f'average: {LOG[run][mode]["final_elbo"]}\n')

      try:
          mir_worked_frac = mir_success/ (mir_tries)
          logging_per_task(wandb, LOG, run, mode, 'final_mir_worked_frac', task, mir_worked_frac)
          print('mir worked \n', mir_worked_frac)
      except:
          pass

    '''

In [27]:
import importlib
#importlib.reload(classifier)
importlib.reload(stable_diffusion)

<module 'stable_diffusion.stable_diffusion' from '/Users/laszlofreund/PycharmProjects/continual-learning-ait/stable_diffusion/stable_diffusion.py'>

In [ ]:
agent = Agent(params)
for r in range(agent.params["n_runs"]):
  agent.state["run"] = r
  run(agent)

h (None, 4, 4, 320)
emb (None, 320)
h (None, 4, 4, 320)
emb (None, 320)
h (None, 2, 2, 640)
emb (None, 640)
h (None, 2, 2, 640)
emb (None, 640)
h (None, 1, 1, 1280)
emb (None, 1280)
h (None, 1, 1, 1280)
emb (None, 1280)
h (None, 1, 1, 1280)
emb (None, 1280)
h (None, 1, 1, 1280)
emb (None, 1280)
h (None, 1, 1, 1280)
emb (None, 1280)
h (None, 1, 1, 1280)
emb (None, 1280)
h (None, 1, 1, 1280)
emb (None, 1280)
h (None, 2, 2, 640)
emb (None, 640)
h (None, 2, 2, 640)
emb (None, 640)
h (None, 2, 2, 640)
emb (None, 640)
h (None, 4, 4, 320)
emb (None, 320)
h (None, 4, 4, 320)
emb (None, 320)
h (None, 4, 4, 320)
emb (None, 320)
(64, 32, 32, 3)


2023-04-17 18:53:55.705402: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
 11  12:   0%|          | 0/12 [00:00<?, ?it/s]

h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
Generator loss:  tf.Tensor(
[ -4.45275    -5.586276   -4.9296036  -8.510807   -5.4032583  -6.5243263
  -8.680038   -7.384837  -11.651019   -2.74195    -4.871292   -4.452985
  -8.10217    -4.6582546  -5.7108917  -5.455018   -3.7294602  -5.3113413
  -6.531668   -5.508216   -8.867166  -11.183591   -4.8973145  -4.5599065
  -4.6262465  -7.665973   -6.432064   -6.42487    -3.3591285  -6.4161077
  -7.5616703  -7.5428066  -4.1183147  -5.513752   

 10  11:   8%|▊         | 1/12 [00:08<01:33,  8.48s/it]

h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
Generator loss:  tf.Tensor(
[ -4.6866074  -5.823448   -5.085408   -8.677839   -5.6463385  -6.7656965
  -8.909419   -7.530851  -11.960329   -2.8714485  -5.0729084  -4.6722603
  -8.218417   -4.8162284  -5.976713   -5.6841965  -3.867021   -5.5487895
  -6.717581   -5.7398276  -9.105291  -11.205311   -5.1211143  -4.8057494
  -4.867646   -7.687046   -6.6853485  -6.679387   -3.5454514  -6.6238194
  -7.865712   -7.7798347  -4.335488   -5.7879543 

  9  10:  17%|█▋        | 2/12 [00:12<00:59,  6.00s/it]

h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 1, 1, 1280)
emb (64, 1280)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 2, 2, 640)
emb (64, 640)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
h (64, 4, 4, 320)
emb (64, 320)
Generator loss:  tf.Tensor(
[ -4.712513   -5.859616   -5.115799   -8.699662   -5.6882076  -6.806996
  -8.909665   -7.5601287 -12.042876   -2.8969302  -5.091666   -4.7083187
  -8.233217   -4.8488736  -6.0188026  -5.7157516  -3.8974576  -5.5788608
  -6.7323484  -5.776719   -9.097996  -11.171671   -5.1556873  -4.838076
  -4.916125   -7.645625   -6.7157125  -6.713606   -3.5851028  -6.6458654
  -7.9027896  -7.807622   -4.370075   -5.835843   -

In [ ]:
''' final eval

n_runs = agent.params["n_runs"]

final_accs = [LOG[x]['final_acc'] for x in range(n_runs)]
final_acc_avg = np.mean(final_accs)
final_acc_se = np.std(final_accs) / np.sqrt(n_runs)

# forgetting
final_forgets = [LOG[x]['final_forget'] for x in range(n_runs)]
final_forget_avg = np.mean(final_forgets)
final_forget_se = np.std(final_forgets) / np.sqrt(n_runs)

# VAE loss
final_elbos = [LOG[x]['final_elbo'] for x in range(n_runs)]
final_elbo_avg = np.mean(final_elbos)
final_elbo_se = np.std(final_elbos) / np.sqrt(n_runs)

# MIR worked
try:
    final_mir_worked_frac = [LOG[x]['final_mir_worked_frac'] for x in range(n_runs)]
    final_mir_worked_avg = np.mean(final_mir_worked_frac)
except:
    pass

print(f'\nFinal Accuracy: {final_acc_avg:.3f} +/- {final_acc_se:.3f}')
print(f'\nFinal Forget: {final_forget_avg:.3f} +/- {final_forget_se:.3f}')
print(f'\nFinal ELBO: {final_elbo_avg:.3f} +/- {final_elbo_se:.3f}')
'''